# EM# GPIO Tests

## Introduction

Here we will make some tests to use the GPIO ports available in the EM#.

The idea is to be able to select different channels connected to a patch panel, throught a multiplexor, and be able to change them depending on the mode that is configured.


## Connection

Let's start openning a socket to the EM#.

In [1]:
import socket

class EMSocket(socket.socket):
    FAMILY = socket.AF_INET
    S_TYPE = socket.SOCK_STREAM
    args = [FAMILY, S_TYPE]
    def __init__(self, em_host, em_port=5025, *args, **kwargs):
        socket.socket.__init__(self, *args, **kwargs)
        self.em_host = em_host
        self.em_port = em_port
    
    def __enter__(self):
        self.connect((self.em_host, self.em_port))
        self.settimeout(0.1)
        return self
    
    def __exit__(self, *args, **kwargs):
        # print args
        # print kwargs
        self.close()


In [2]:
EM = 'w-kitslab-em2-0'
MODE = 2

In [4]:
with EMSocket('w-kitslab-em2-0') as ems:
    ems.send("*IDN?\n")
    data = ""
    while not data.endswith('\n'):
        recived = ems.recv(10240)
        data += recived
    print recived
    print data.endswith('\n')

ALBASYNCHROTRON,Electrometer2,000000007, 1.10

True


In [36]:
MODE = 2
with EMSocket(EM) as ems:
    ems.send("IOPO05:CONF 0;IOPO06:CONF 0;IOPO07:CONF 0\n")
    a0 = MODE & 1 > 0
    a1 = MODE & 2 > 0
    a2 = MODE & 4 > 0
    print a0, a1, a2
    cmd = "IOPO05:VALU %d;IOPO06:VALU %d;IOPO07:VALU %d\n" % (a0, a1, a2)
    ems.send(cmd)

False True False


In [2]:
with EMSocket(EM) as ems:
    ems.send("IOPO05:CONF 0;\n")
    a0 = True
    cmd = "IOP05:VALU %d;\n" % a0
    ems.send(cmd)

NameError: name 'EMSocket' is not defined

# Testing the ports

The following code will test one by one the different modes available.
We can also see how the leds are changed in the patch panel.

In [55]:
import time

def send_mode(mode):
    with EMSocket(EM) as ems:
        ems.send("IOPO05:CONF 0;IOPO06:CONF 0;IOPO07:CONF 0\n")
        a0 = mode & 1 > 0
        a1 = mode & 2 > 0
        a2 = mode & 4 > 0
        print a0, a1, a2
        time.sleep(1)
        cmd = "IOPO05:VALU %d;IOPO06:VALU %d;IOPO07:VALU %d\n" % (a0, a1, a2)
        ems.send(cmd)

for i in range(8):
    print "Sending mode %d"%i
    send_mode(i)
    time.sleep(3)

Sending mode 0
False False False
Sending mode 1
True False False
Sending mode 2
False True False
Sending mode 3
True True False
Sending mode 4
False False True
Sending mode 5
True False True
Sending mode 6
False True True
Sending mode 7
True True True
